In [1]:
!pip install --upgrade tensorflow --user
!pip install tf-quant-finance
!pip install QuantLib-Python

In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import tf_quant_finance as tff 
import tensorflow as tf
import functools
import pandas as pd
import time

# Shortcut alias
pde = tff.math.pde
option_price = tff.black_scholes.option_price
implied_vol = tff.black_scholes.implied_vol

In [3]:
tf_expiry = tff.datetime.daycount_actual_actual_isda(
          start_date=tff.datetime.dates_from_tuples([(2022, 3, 24)]), 
          end_date=tff.datetime.dates_from_tuples([(2023, 3, 24)]),
          dtype=tf.float64)
tf_spot = tf.constant(36.0,tf.float64, shape=[2, 1])
tf_strike = tf.constant([36.2, 34],tf.float64, shape=[2, 1])
tf_volatility = tf.constant(0.20,tf.float64, shape=[2, 1])
tf_discount_rates = tf.constant(0.02,tf.float64, shape=[2, 1])
tf_dividend_rates = tf.constant(0.07,tf.float64, shape=[2, 1])
tf_is_call = tf.constant(False, dtype=tf.bool, shape=[2, 1])

print(tf_strike)

tf.Tensor(
[[36.2]
 [34. ]], shape=(2, 1), dtype=float64)


In [4]:
tqf_BS_price = option_price(
      volatilities=tf_volatility,
      strikes=tf_strike,
      expiries=tf_expiry,
      spots=tf_spot,
      discount_rates=tf_discount_rates,
      dividend_rates=tf_dividend_rates,
      is_call_options=tf_is_call
      ).numpy()[0]
print(tqf_BS_price[0])

3.813573900567217


In [5]:
price_fn = functools.partial(tff.black_scholes.option_price,
        volatilities=tf_volatility,
        strikes=tf_strike,
        expiries=tf_expiry,
        spots=tf_spot,
        discount_rates=tf_discount_rates,
        dividend_rates=tf_dividend_rates,
        is_call_options=tf_is_call,
        dtype=np.float64)
tff_BS_delta = tff.math.fwd_gradient(lambda x: price_fn(spots=x), tf_spot)
tff_BS_gamma = tff.math.fwd_gradient(lambda x: tff.math.fwd_gradient(lambda x: price_fn(spots=x), tf_spot), tf_spot)
tff_BS_rho = tff.math.fwd_gradient(lambda x: price_fn(discount_rates=x), tf_discount_rates)
tff_BS_vega = tff.math.fwd_gradient(lambda x: price_fn(volatilities=x), tf_volatility)
tff_BS_theta = -tff.math.fwd_gradient(lambda x: price_fn(expiries=x), tf_expiry)

print('tff_BS_delta', tff_BS_delta[0,0].numpy())
print('tff_BS_gamma', tff_BS_gamma[0,0].numpy())
print('tff_BS_rho', tff_BS_rho[0,0].numpy())
print('tff_BS_vega', tff_BS_vega[0,0].numpy())
print('tff_BS_theta', tff_BS_theta[0,0].numpy())
print('tff_BS_theta', tff_BS_theta)

tff_BS_delta -0.5319503119033676
tff_BS_gamma 0.050853396486462255
tff_BS_rho -22.963785129088453
tff_BS_vega 13.181200369291014
tff_BS_theta -2.1993591203438188
tff_BS_theta tf.Tensor(
[[-2.19935912]
 [-2.02439727]], shape=(2, 1), dtype=float64)


In [6]:
# tf.function decorator makes the function faster in graph mode.
#@tf.function
def vanilla_put_option(number_grid_points,
                    num_steps,
                    strike,
                    volatility,
                    risk_free_rate,
                    dividend_rates,
                    expiry,
                    spot,
                    european_exercise=False,
                    nb_bound=3.0,
                    dtype=tf.float64):
  """ Computes American Call options prices.

  Args:
    number_grid_points: A Python int. Number of grid points for the finite
      difference scheme.
    time_delta: A Python float. Grid time discretization parameter.
    strike: A real `Tensor` of shape `(number_of_options, 1)`.
      Represents the strikes of the underlying American options. 
    volatility: A real `Tensor` of shape `(number_of_options, 1)`.
      Represents the volatilities of the underlying American options. 
    risk_free_rate: A real `Tensor` of shape `(number_of_options, 1)`.
      Represents the risk-free interest rates associated with the underlying
      American options.
    expiry: A Python float. Expiry date of the options. If the options
      have different expiries, volatility term has to adjusted to
      make expiries the same.
    dtype: Optional `tf.dtype` used to assert dtype of the input `Tensor`s.

  Returns:
    A tuple of the estimated option prices of shape
    `(number_of_options, number_grid_points)` and the corresponding `Tensor` 
    of grid locations of shape `(number_grid_points,)`.
  """
  # Define the coordinate grid
#   s_min = spot/nb_bound #0.01
#   s_max = nb_bound*spot #300.
#   print(s_min, s_max)
  s_min = 0.01
  s_max = 300.

  grid = pde.grids.uniform_grid(minimums=[s_min],
                                maximums=[s_max],
                                sizes=[number_grid_points],
                                dtype=dtype)

  # Define the values grid for the final condition
  s = grid[0]
#   final_values_grid = tf.nn.relu(s - strike)
  final_values_grid = tf.nn.relu(strike-s)

  # Define the PDE coefficient functions
  def second_order_coeff_fn(t, grid):
    del t
    s = grid[0]
    return [[volatility ** 2 * s ** 2 / 2]]

  def first_order_coeff_fn(t, grid):
    del t
    s = grid[0]
    return [(risk_free_rate-dividend_rates) * s]

  def zeroth_order_coeff_fn(t, grid):
    del t, grid
    return -risk_free_rate

  # Define the boundary conditions
  @pde.boundary_conditions.dirichlet
  def lower_boundary_fn(t, grid):
    del t, grid
    return tf.constant(0.0, dtype=dtype)

  @pde.boundary_conditions.dirichlet
  def upper_boundary_fn(t, grid):
    del grid
#     return tf.squeeze(s_max - strike * tf.exp(-(risk_free_rate-dividend_rates) * (expiry - t)))
    return tf.squeeze(strike * tf.exp(-(risk_free_rate-dividend_rates) * (expiry - t)) - s_max)

  # In order to price American option one needs to set option values to 
  # V(x) := max(V(x), max(x - strike, 0)) after each iteration
  def values_transform_fn(t, grid, values):
    del t
    s = grid[0]
    if european_exercise==False:
#         values_floor = tf.nn.relu(s-strike)
        values_floor = tf.nn.relu(strike-s)
        return grid, tf.maximum(values, values_floor)
    else:
        return grid, values

  # Solve
  estimate_values, estimate_grid, _, _ = \
    pde.fd_solvers.solve_backward(
      start_time=expiry,
      end_time=0,
      values_transform_fn=values_transform_fn,
      coord_grid=grid,
      values_grid=final_values_grid,
      num_steps=num_steps,
      boundary_conditions=[(lower_boundary_fn, upper_boundary_fn)],
      second_order_coeff_fn=second_order_coeff_fn,
      first_order_coeff_fn=first_order_coeff_fn,
      zeroth_order_coeff_fn=zeroth_order_coeff_fn,
      dtype=dtype
    )
  return estimate_values, estimate_grid[0]

@tf.function(jit_compile=False,
             input_signature=[
                              tf.TensorSpec([], dtype=tf.int32),
                              tf.TensorSpec([], dtype=tf.int32),
                              tf.TensorSpec([None,1], dtype=tf.float64),
                              tf.TensorSpec([None,1], dtype=tf.float64),
                              tf.TensorSpec([None,1], dtype=tf.float64),
                              tf.TensorSpec([None,1], dtype=tf.float64),
                              tf.TensorSpec([None], dtype=tf.float64),
                              tf.TensorSpec([None,1], dtype=tf.float64),
                              tf.TensorSpec([], dtype=tf.float64),
                              tf.TensorSpec([], dtype=tf.float64)
                     ])
def tqf_price(number_grid_points,
                    num_steps,
                    strike,
                    volatility,
                    risk_free_rate,
                    dividend_rates,
                    expiry,
                    spot,
                    european_exercise=False,
                    nb_bound=3.0):
    # print('number_grid_points', number_grid_points)
    # print('strike', strike)
    # print('spot', spot)
    estimate, grid_locations = vanilla_put_option(
    num_steps=num_steps,
    expiry=expiry,
    number_grid_points=number_grid_points,
    volatility=volatility,
    risk_free_rate=risk_free_rate,
    dividend_rates=dividend_rates,
    strike=strike,
    spot=spot,
    european_exercise=european_exercise,
    nb_bound=3.0,
    dtype=tf.float64)

#     estimate = estimate.numpy()
#     grid_locations = grid_locations.numpy()
#     tqf_euro_price_pde = interp1d(grid_locations, estimate)(spot)
    # print('spot', spot)
    # print('grid_locations', grid_locations)
    # print('estimate', estimate)
    tqf_euro_price_pde = tff.math.interpolation.linear.interpolate(spot, grid_locations, estimate, dtype=tf.float64)
    return tqf_euro_price_pde


number_grid_points = 1024
num_steps = 200


print('tf_strike', tf_strike)
print('tf_spot', tf_spot)

tqf_price_pde = tqf_price(number_grid_points=number_grid_points,
                    num_steps=num_steps,
                    strike=tf_strike,
                    volatility=tf_volatility,
                    risk_free_rate=tf_discount_rates,
                    dividend_rates=tf_dividend_rates,
                    expiry=tf_expiry,
                    spot=tf_spot,
                    european_exercise=True,
                    nb_bound=3.0)
print('tqf_price_pde', tqf_price_pde[0,0].numpy(), tqf_BS_price[0])
print('tqf_price_pde', tqf_price_pde.numpy(), tqf_BS_price[0])

tqf_price_pde_spotplus = tqf_price(number_grid_points=number_grid_points,
                    num_steps=num_steps,
                    strike=tf_strike,
                    volatility=tf_volatility,
                    risk_free_rate=tf_discount_rates,
                    dividend_rates=tf_dividend_rates,
                    expiry=tf_expiry,
                    spot=tf_spot*1.01,
                    european_exercise=False,
                    nb_bound=3.0)
print('tqf_price_pde_spotplus', tqf_price_pde_spotplus[0,0].numpy())
print('tqf_delta_pde_bump', (tqf_price_pde_spotplus[0,0].numpy() - tqf_price_pde[0,0].numpy())/(0.01*tf_spot)[0,0].numpy(), tff_BS_delta[0,0].numpy()) 

tf_strike tf.Tensor(
[[36.2]
 [34. ]], shape=(2, 1), dtype=float64)
tf_spot tf.Tensor(
[[36.]
 [36.]], shape=(2, 1), dtype=float64)
tqf_price_pde 3.8139554737011774 3.813573900567217
tqf_price_pde [[3.81395547]
 [2.54625872]] 3.813573900567217
tqf_price_pde_spotplus 3.625419039639332
tqf_delta_pde_bump -0.5237123168384589 -0.5319503119033676


In [7]:
@tf.function(jit_compile=False,
             input_signature=[
                              tf.TensorSpec([], dtype=tf.int32),
                              tf.TensorSpec([], dtype=tf.int32),
                              tf.TensorSpec([None,1], dtype=tf.float64),
                              tf.TensorSpec([None,1], dtype=tf.float64),
                              tf.TensorSpec([None,1], dtype=tf.float64),
                              tf.TensorSpec([None,1], dtype=tf.float64),
                              tf.TensorSpec([None], dtype=tf.float64),
                              tf.TensorSpec([None,1], dtype=tf.float64),
                              tf.TensorSpec([], dtype=tf.float64),
                              tf.TensorSpec([], dtype=tf.float64)
                     ])
def delta_fn_xla(number_grid_points,
                    num_steps,
                    strike,
                    volatility,
                    risk_free_rate,
                    dividend_rates,
                    expiry,
                    spot,
                    european_exercise=False,
                    nb_bound=3.0):
    fn = lambda spot: tqf_price(number_grid_points=number_grid_points,
                    num_steps=num_steps,
                    strike=strike,
                    volatility=volatility,
                    risk_free_rate=risk_free_rate,
                    dividend_rates=dividend_rates,
                    expiry=expiry,
                    spot=spot,
                    european_exercise=european_exercise,
                    nb_bound=nb_bound)
    return tff.math.fwd_gradient(fn, spot)
    
@tf.function(jit_compile=True)
def vega_fn_xla(number_grid_points,
                    num_steps,
                    strike,
                    volatility,
                    risk_free_rate,
                    dividend_rates,
                    expiry,
                    spot,
                    european_exercise=False,
                    nb_bound=3.0):
    fn = lambda volatility: tqf_price(number_grid_points=number_grid_points,
                    num_steps=num_steps,
                    strike=strike,
                    volatility=volatility,
                    risk_free_rate=risk_free_rate,
                    dividend_rates=dividend_rates,
                    expiry=expiry,
                    spot=spot,
                    european_exercise=european_exercise,
                    nb_bound=nb_bound)
    return tff.math.fwd_gradient(fn, volatility)

@tf.function(jit_compile=True)
def rho_fn_xla(number_grid_points,
                    num_steps,
                    strike,
                    volatility,
                    risk_free_rate,
                    dividend_rates,
                    expiry,
                    spot,
                    european_exercise=False,
                    nb_bound=3.0):
    fn = lambda risk_free_rate: tqf_price(number_grid_points=number_grid_points,
                    num_steps=num_steps,
                    strike=strike,
                    volatility=volatility,
                    risk_free_rate=risk_free_rate,
                    dividend_rates=dividend_rates,
                    expiry=expiry,
                    spot=spot,
                    european_exercise=european_exercise,
                    nb_bound=nb_bound)
    return tff.math.fwd_gradient(fn, risk_free_rate)

@tf.function(jit_compile=True)
def theta_fn_xla(number_grid_points,
                    num_steps,
                    strike,
                    volatility,
                    risk_free_rate,
                    dividend_rates,
                    expiry,
                    spot,
                    european_exercise=False,
                    nb_bound=3.0):
    fn = lambda expiry: tqf_price(number_grid_points=number_grid_points,
                    num_steps=num_steps,
                    strike=strike,
                    volatility=volatility,
                    risk_free_rate=risk_free_rate,
                    dividend_rates=dividend_rates,
                    expiry=expiry,
                    spot=spot,
                    european_exercise=european_exercise,
                    nb_bound=nb_bound)
    return tff.math.fwd_gradient(fn, expiry)                   


# print('forward theta', -tff.math.fwd_gradient(lambda x: price_fn(expiry=x), tf_expiry))
# print('backward theta', -tff.math.gradients(lambda x: price_fn(expiry=x), tf_expiry))
# print('backward theta', -tff.math.gradients(lambda x: price_fn(expiry=x), tf_expiry, 
#                                             output_gradients=tf.constant([1, 0], dtype=tf.float64, shape=[2, 1])))
# print('backward theta', -tff.math.gradients(lambda x: price_fn(expiry=x), tf_expiry, 
#                                             output_gradients=tf.constant([0, 1], dtype=tf.float64, shape=[2, 1])))


        
tff_delta_pde = delta_fn_xla(number_grid_points=number_grid_points,
                    num_steps=num_steps,
                    strike=tf_strike,
                    volatility=tf_volatility,
                    risk_free_rate=tf_discount_rates,
                    dividend_rates=tf_dividend_rates,
                    expiry=tf_expiry,
                    spot=tf_spot,
                    european_exercise=True,
                    nb_bound=3.0)
# tff_vega_pde = tff.math.gradients(lambda x: price_fn(volatility=x), tf_volatility)
# tff_rho_pde = tff.math.gradients(lambda x: price_fn(risk_free_rate=x), tf_discount_rates)
# tff_theta_pde = -tff.math.gradients(lambda x: price_fn(expiry=x), tf_expiry)
print('tff_delta_pde', tff_delta_pde[0,0].numpy(), tff_BS_delta[0,0].numpy() )
# print('tff_vega_pde', tff_vega_pde[0,0].numpy(), tff_BS_vega[0,0].numpy() )
# print('tff_rho_pde', tff_rho_pde[0,0].numpy(), tff_BS_rho[0,0].numpy() )
# print('tff_theta_pde', tff_vega_pde)
# print('tff_theta_pde', tff_theta_pde)

tff_delta_pde -0.5353565511214893 -0.5319503119033676


In [8]:
device = "/cpu:0"
with tf.device(device):
    # batch_of_options = [5, 10, 100, 150, 200, 250, 300, 350, 400, 450, 500, 600, 700, 800, 900, 1000, 10000]
    batch_of_options = [10]
    for number_of_options in batch_of_options:
        filename = "https://raw.githubusercontent.com/arthurpham/google_colab/main/data/American_Option_Black_Scholes_GenerateCSV_{}.csv".format(number_of_options)
        option_inputs_df = pd.read_csv(filename)

        strikes = option_inputs_df['strike']

        tf_volatility = tf.constant(option_inputs_df['volatility'][0], dtype=tf.float64, shape=[len(strikes), 1]  )
        tf_discount_rates = tf.constant(option_inputs_df['risk_free_rate'][0], dtype=tf.float64, shape=[len(strikes), 1]  )
        tf_dividend_rates = tf.constant(option_inputs_df['risk_free_rate'][0], dtype=tf.float64, shape=[len(strikes), 1]  )
        tf_strike = tf.constant(option_inputs_df['strike'], dtype=tf.float64, shape=[len(strikes), 1]  )
        tf_expiry = tf.constant(option_inputs_df['expiry'][0], dtype=tf.float64, shape=[1] )
        tf_spot = tf.constant(option_inputs_df['strike'][0], dtype=tf.float64, shape=[len(strikes), 1]  )

        t = time.time()
        tqf_price_pde = tqf_price(number_grid_points=number_grid_points,
                        num_steps=num_steps,
                        strike=tf_strike,
                        volatility=tf_volatility,
                        risk_free_rate=tf_discount_rates,
                        dividend_rates=tf_dividend_rates,
                        expiry=tf_expiry,
                        spot=tf_spot,
                        european_exercise=True,
                        nb_bound=3.0)

        price_fn = functools.partial(tqf_price,
                number_grid_points=number_grid_points,
                num_steps=num_steps,
                strike=tf_strike,
                volatility=tf_volatility,
                risk_free_rate=tf_discount_rates,
                dividend_rates=tf_dividend_rates,
                expiry=tf_expiry,
                spot=tf_spot,
                european_exercise=True,
                nb_bound=3.0)

        tff_delta_pde = tff.math.gradients(lambda x: price_fn(spot=x), tf_spot)
        tff_vega_pde = tff.math.gradients(lambda x: price_fn(volatility=x), tf_volatility)
        tff_rho_pde = tff.math.gradients(lambda x: price_fn(risk_free_rate=x), tf_discount_rates)
        tff_theta_pde = []
        for i in range(0, len(strikes)):
            output_gradient = tf.Variable(tf.constant(0.0, dtype=tf.float64, shape=[len(strikes), 1]))
            output_gradient[i, 0].assign(1.0)
            tff_theta_pde.append(-tff.math.gradients(lambda x: price_fn(expiry=x), tf_expiry, output_gradients=output_gradient).numpy())

        time_tqf = time.time() - t

        tqf_options_per_second = number_of_options / time_tqf
        print('------------------------')
        print('TQF CPU {} options'.format(number_of_options))
        print('wall time: ', time_tqf)
        print('options per second: ', tqf_options_per_second)
        print('------------------------')
        print('price', tqf_price_pde[:10, 0].numpy())
        print('delta', tff_delta_pde[:10, 0].numpy())
        print('vega', tff_vega_pde[:10, 0].numpy())
        print('rho', tff_rho_pde[:10, 0].numpy())
        print('theta', tff_theta_pde)

------------------------
TQF CPU 10 options
wall time:  67.00861430168152
options per second:  0.149234544009203
------------------------
price [12.41736972 92.79272379 79.91626123 25.81403894 23.50151285 23.61974543
 33.0870351  51.89128652 43.79193454 32.032111  ]
delta [-0.44096027 -0.90858834 -0.88339451 -0.61082056 -0.58698732 -0.58825278
 -0.67526503 -0.79024046 -0.74798439 -0.66679029]
vega [11.65712933  4.64065997  6.2217599  13.96377161 13.95787491 13.96087984
 13.46277426 10.61544448 11.96598429 13.57214016]
rho [ -61.23522924 -244.50194684 -219.67277992 -100.15518557  -93.93691149
  -94.25910985 -118.75436623 -162.13859438 -144.11393258 -116.13610791]
theta [array([-1.47170685]), array([1.05998028]), array([0.58743141]), array([-1.55893842]), array([-1.60134185]), array([-1.59956989]), array([-1.34947317]), array([-0.58055275]), array([-0.92992762]), array([-1.3852311])]


In [9]:
print(tqf_price.pretty_printed_concrete_signatures())

tqf_price(number_grid_points, num_steps, strike, volatility, risk_free_rate, dividend_rates, expiry, spot, european_exercise, nb_bound)
  Args:
    number_grid_points: int32 Tensor, shape=()
    num_steps: int32 Tensor, shape=()
    strike: float64 Tensor, shape=(None, 1)
    volatility: float64 Tensor, shape=(None, 1)
    risk_free_rate: float64 Tensor, shape=(None, 1)
    dividend_rates: float64 Tensor, shape=(None, 1)
    expiry: float64 Tensor, shape=(None,)
    spot: float64 Tensor, shape=(None, 1)
    european_exercise: float64 Tensor, shape=()
    nb_bound: float64 Tensor, shape=()
  Returns:
    float64 Tensor, shape=(None, 1)


In [10]:
device = "/gpu:0"
with tf.device(device):
    # batch_of_options = [5, 10, 100, 150, 200, 250, 300, 350, 400, 450, 500, 600, 700, 800, 900, 1000, 10000]
    batch_of_options = [10, 10]
    for number_of_options in batch_of_options:
        filename = "https://raw.githubusercontent.com/arthurpham/google_colab/main/data/American_Option_Black_Scholes_GenerateCSV_{}.csv".format(number_of_options)
        option_inputs_df = pd.read_csv(filename)

        strikes = option_inputs_df['strike']

        tf_volatility = tf.constant(option_inputs_df['volatility'][0], dtype=tf.float64, shape=[len(strikes), 1]  )
        tf_discount_rates = tf.constant(option_inputs_df['risk_free_rate'][0], dtype=tf.float64, shape=[len(strikes), 1]  )
        tf_dividend_rates = tf.constant(option_inputs_df['risk_free_rate'][0], dtype=tf.float64, shape=[len(strikes), 1]  )
        tf_strike = tf.constant(option_inputs_df['strike'], dtype=tf.float64, shape=[len(strikes), 1]  )
        tf_expiry = tf.constant(option_inputs_df['expiry'][0], dtype=tf.float64, shape=[1] )
        tf_spot = tf.constant(option_inputs_df['strike'][0], dtype=tf.float64, shape=[len(strikes), 1]  )

        t = time.time()
        tqf_price_pde = tqf_price(number_grid_points=number_grid_points,
                        num_steps=num_steps,
                        strike=tf_strike,
                        volatility=tf_volatility,
                        risk_free_rate=tf_discount_rates,
                        dividend_rates=tf_dividend_rates,
                        expiry=tf_expiry,
                        spot=tf_spot,
                        european_exercise=True,
                        nb_bound=3.0)

        price_fn = functools.partial(tqf_price,
                number_grid_points=number_grid_points,
                num_steps=num_steps,
                strike=tf_strike,
                volatility=tf_volatility,
                risk_free_rate=tf_discount_rates,
                dividend_rates=tf_dividend_rates,
                expiry=tf_expiry,
                spot=tf_spot,
                european_exercise=True,
                nb_bound=3.0)

        tff_delta_pde = tff.math.gradients(lambda x: price_fn(spot=x), tf_spot)
        tff_vega_pde = tff.math.gradients(lambda x: price_fn(volatility=x), tf_volatility)
        tff_rho_pde = tff.math.gradients(lambda x: price_fn(risk_free_rate=x), tf_discount_rates)
        tff_theta_pde = []
        for i in range(0, len(strikes)):
            output_gradient = tf.Variable(tf.constant(0.0, dtype=tf.float64, shape=[len(strikes), 1]))
            output_gradient[i, 0].assign(1.0)
            tff_theta_pde.append(-tff.math.gradients(lambda x: price_fn(expiry=x), tf_expiry, output_gradients=output_gradient).numpy())

        time_tqf = time.time() - t

        tqf_options_per_second = number_of_options / time_tqf
        print('------------------------')
        print('TQF GPU {} options'.format(number_of_options))
        print('wall time: ', time_tqf)
        print('options per second: ', tqf_options_per_second)
        print('------------------------')
        print('price', tqf_price_pde[:10, 0].numpy())
        print('delta', tff_delta_pde[:10, 0].numpy())
        print('vega', tff_vega_pde[:10, 0].numpy())
        print('rho', tff_rho_pde[:10, 0].numpy())
        print('theta', tff_theta_pde)

------------------------
TQF GPU 10 options
wall time:  62.10876202583313
options per second:  0.16100787833833594
------------------------
price [12.41736972 92.79272379 79.91626123 25.81403894 23.50151285 23.61974543
 33.0870351  51.89128652 43.79193454 32.032111  ]
delta [-0.44096027 -0.90858834 -0.88339451 -0.61082056 -0.58698732 -0.58825278
 -0.67526503 -0.79024046 -0.74798439 -0.66679029]
vega [11.65712933  4.64065997  6.2217599  13.96377161 13.95787491 13.96087984
 13.46277426 10.61544448 11.96598429 13.57214016]
rho [ -61.23522924 -244.50194684 -219.67277992 -100.15518557  -93.93691149
  -94.25910985 -118.75436623 -162.13859438 -144.11393258 -116.13610791]
theta [array([-1.47170685]), array([1.05998028]), array([0.58743141]), array([-1.55893842]), array([-1.60134185]), array([-1.59956989]), array([-1.34947317]), array([-0.58055275]), array([-0.92992762]), array([-1.3852311])]
------------------------
TQF GPU 10 options
wall time:  54.5565447807312
options per second:  0.1832960

In [11]:
device = "/gpu:0"
with tf.device(device):
    # batch_of_options = [5, 10, 100, 150, 200, 250, 300, 350, 400, 450, 500, 600, 700, 800, 900, 1000, 10000]
    batch_of_options = [10, 100, 1000, 10000, 10, 100, 1000, 10000]
    for number_of_options in batch_of_options:
        filename = "https://raw.githubusercontent.com/arthurpham/google_colab/main/data/American_Option_Black_Scholes_GenerateCSV_{}.csv".format(number_of_options)
        option_inputs_df = pd.read_csv(filename)

        strikes = option_inputs_df['strike']

        tf_volatility = tf.constant(option_inputs_df['volatility'][0], dtype=tf.float64, shape=[len(strikes), 1]  )
        tf_discount_rates = tf.constant(option_inputs_df['risk_free_rate'][0], dtype=tf.float64, shape=[len(strikes), 1]  )
        tf_dividend_rates = tf.constant(option_inputs_df['risk_free_rate'][0], dtype=tf.float64, shape=[len(strikes), 1]  )
        tf_strike = tf.constant(option_inputs_df['strike'], dtype=tf.float64, shape=[len(strikes), 1]  )
        tf_expiry = tf.constant(option_inputs_df['expiry'][0], dtype=tf.float64, shape=[1] )
        tf_spot = tf.constant(option_inputs_df['strike'][0], dtype=tf.float64, shape=[len(strikes), 1]  )

        t = time.time()
        tqf_price_pde = tqf_price(number_grid_points=number_grid_points,
                        num_steps=num_steps,
                        strike=tf_strike,
                        volatility=tf_volatility,
                        risk_free_rate=tf_discount_rates,
                        dividend_rates=tf_dividend_rates,
                        expiry=tf_expiry,
                        spot=tf_spot,
                        european_exercise=True,
                        nb_bound=3.0)

        time_tqf = time.time() - t

        tqf_options_per_second = number_of_options / time_tqf
        print('------------------------')
        print('TQF GPU {} options'.format(number_of_options))
        print('wall time: ', time_tqf)
        print('options per second: ', tqf_options_per_second)
        print('------------------------')
        print('price', tqf_price_pde[:10, 0].numpy())

------------------------
TQF GPU 10 options
wall time:  0.5494236946105957
options per second:  18.200889583197725
------------------------
price [12.41736972 92.79272379 79.91626123 25.81403894 23.50151285 23.61974543
 33.0870351  51.89128652 43.79193454 32.032111  ]
------------------------
TQF GPU 100 options
wall time:  0.6024224758148193
options per second:  165.996462639849
------------------------
price [10.34879861 61.04464704 31.52063324 48.98336542 87.31300191 26.05500285
 39.98319504 72.50599112 24.38995847 13.19918096]
------------------------
TQF GPU 1000 options
wall time:  2.005411386489868
options per second:  498.6508038883384
------------------------
price [ 1.77652382 35.66277122 68.75424105 54.68768024 62.11838296 47.34641651
 50.6944191  66.38015664  6.62796853 30.41723658]
------------------------
TQF GPU 10000 options
wall time:  17.672589540481567
options per second:  565.8480313308689
------------------------
price [13.4940461  11.28811224  4.60529981 29.115479